In [1]:
from google.colab import drive
import os

import pandas as pd
import numpy as np
!pip install xlsxwriter

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/2 - [FINANCEIRO]/[2023]/Extratos')
os.listdir()

Mounted at /content/gdrive


['[PLANEJAMENTO]', 'Fotos Produtos Lojinha', 'Extratos']

In [ ]:
def month_mapper(month):
     keys = {
         "01": "JANEIRO",
         "02": "FEVEREIRO",
         "03": "MARÇO",
         "04": "ABRIL",
         "05": "MAIO",
         "06": "JUNHO",
         "07": "JULHO",
         "08": "AGOSTO",
         "09": "SETEMBRO",
         "10": "OUTUBRO",
         "11": "NOVEMBRO",
         "12": "DEZEMBRO"
     }

     return keys[month]

def number_formatter(number):
    number = float(number)
    return "{:,.2f}".format(number).replace(',', '#').replace('.', ',').replace('#', '.')

In [58]:
# abre o arquivo csv
arquivo = '2023-03.csv'
df = pd.read_csv(arquivo, encoding='utf-8-sig', sep=',', dtype=str)

# formata os valores com separador de milesimo para . e de decimal para , 
df['Valor'] = df['Valor'].astype(float)
df['Valor'] = df['Valor'].apply(lambda x: number_formatter(x))

In [ ]:
# Divide os valores para a coluna entrada e saida
df['Entrada R$'] = df.apply(lambda x: x['Valor'] if x['Tipo Transação'] == 'CRÉDITO' else '', axis=1)
df['Saida R$'] = df.apply(lambda x: x['Valor'] if x['Tipo Transação'] == 'DÉBITO' else '', axis=1)
df = df.drop(columns=['Valor', 'Tipo Transação'], axis=1)

# cria coluna historico
df['Historico'] = ''

# cria coluna detalhes concatenando valores da transacao e identificacao
df['Detalhes'] = df[['Transação', 'Identificação']].apply(': '.join, axis=1)
df = df.drop(columns=['Transação', 'Identificação'], axis=1)

# formata df
df = df.reindex(columns=['Data', 'Historico', 'Detalhes', 'Entrada R$', 'Saida R$'])
df = df.sort_values(by = 'Data')

df

In [62]:
out_filename = os.getcwd() + '/' + arquivo.replace('.csv', '.xlsx')
writer = pd.ExcelWriter(out_filename)
df.to_excel(writer, sheet_name='balanço', index=False, header=True)

for column in df:
    column_width = max(df[column].astype(str).map(len).max(), len(column))
    col_idx = df.columns.get_loc(column)
    writer.sheets['balanço'].set_column(col_idx, col_idx, column_width)

writer.save()